# Working with cell-based coadds

Last working weekly: w2024_28

## Load in relevant data

In [1]:
REPO = '/sdf/data/rubin/repo/main/'

from lsst.daf.butler import Butler
from pprint import pprint
import lsst.afw.image as afwImage
from lsst.skymap import Index2D
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

%matplotlib inline

butler = Butler(REPO)
registry = butler.registry

Confirm that local packages are being used, tested with `pipe_tasks` and `drp_tasks`. Need to check this at least until these package versions are in the main stack.

In [2]:
import lsst.pipe.tasks
import lsst.drp.tasks
print(lsst.pipe.tasks.__file__)
print(lsst.drp.tasks.__file__)

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/pipe_tasks/gf226f04e1d+4607d2a8d7/python/lsst/pipe/tasks/__init__.py
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/drp_tasks/g5d9d3f8791+31958832b9/python/lsst/drp/tasks/__init__.py


Define the collection of interest

In [3]:
# collection = 'u/mgorsuch/assemble_cell_coadd_patch_61/20240403T183342Z' # for the patch 61 collection, with 8 detected warps.
collection = 'u/mgorsuch/assemble_cell_coadd_patch_61_d0508/20240509T165409Z' # updated patch 61 collection

## Example butler queries

Check which dataset types are registered in the collection.

In [4]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(execute=False, exact=False):
        print(datasetType)

DatasetType('packages', {}, Packages)
DatasetType('deepCoadd_directWarp', {band, instrument, skymap, day_obs, physical_filter, tract, patch, visit}, ExposureF)
DatasetType('makeDirectWarp_config', {}, Config)
DatasetType('deepCoaddCell', {band, skymap, tract, patch}, MultipleCellCoadd)
DatasetType('deepCoadd_directWarp_noise0', {band, instrument, skymap, day_obs, physical_filter, tract, patch, visit}, MaskedImageF)
DatasetType('makeDirectWarp_metadata', {band, instrument, skymap, day_obs, physical_filter, tract, patch, visit}, TaskMetadata)
DatasetType('makeDirectWarp_log', {band, instrument, skymap, day_obs, physical_filter, tract, patch, visit}, ButlerLogRecords)
DatasetType('assembleCellCoadd_config', {}, Config)
DatasetType('assembleCellCoadd_metadata', {band, skymap, tract, patch}, TaskMetadata)
DatasetType('assembleCellCoadd_log', {band, skymap, tract, patch}, ButlerLogRecords)
DatasetType('deepCoadd_directWarp_maskedFraction', {band, instrument, skymap, day_obs, physical_filter,

See how many objects of `deepCoadd_directWarp` are in the collection

In [5]:
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', physical_filter='HSC-I', collections=collection, instrument='HSC'):
    print(ref.dataId)

{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 61, visit: 1242, band: 'i', day_obs: 20140328, physical_filter: 'HSC-I'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 61, visit: 1242, band: 'i', day_obs: 20140328, physical_filter: 'HSC-I'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 61, visit: 1242, band: 'i', day_obs: 20140328, physical_filter: 'HSC-I'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 61, visit: 1242, band: 'i', day_obs: 20140328, physical_filter: 'HSC-I'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 61, visit: 1242, band: 'i', day_obs: 20140328, physical_filter: 'HSC-I'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 61, visit: 1242, band: 'i', day_obs: 20140328, physical_filter: 'HSC-I'}
{instrument: 'HSC', skymap: 'hsc_rings_cells_v1', tract: 9813, patch: 61, visit: 30490, band: 'i', day_obs: 20150521, physical_filter: 'HSC-I'

## Load in example warp

In [6]:
warp = butler.get('deepCoadd_directWarp',
                   collections = collection,
                   instrument='HSC',
                   skymap = 'hsc_rings_cells_v1',
                   tract = 9813,
                   patch = 61,
                   visit = 30482)

Take a look at the cell coadd structure

In [7]:
coadd = butler.get('deepCoaddCell', 
                     collections=collection, 
                     instrument='HSC', 
                     skymap = 'hsc_rings_cells_v1', 
                     tract = 9813, 
                     patch=61,
                     band='i',)

The cells within the produced coadd can be "stitched" together to produce a single coadd structure. Useful for displaying the image of an entire patch formed by combining all cells.

In [8]:
stitch_coadd = coadd.stitch()

Check the number of cells with inputs. For instance, the patch 37 contains 85 cells with inputs, while patch 61 has all 484 cells with inputs (484=22x22 cells in a patch, with this configuration).

In [9]:
cell_num = len(list(coadd.cells.keys()))
print(cell_num)

484


We can see a list of the available cells with their index information:

In [10]:
cell_list = list(coadd.cells.keys())
print(cell_list)

[Index2D(x=0, y=0), Index2D(x=1, y=0), Index2D(x=2, y=0), Index2D(x=3, y=0), Index2D(x=4, y=0), Index2D(x=5, y=0), Index2D(x=6, y=0), Index2D(x=7, y=0), Index2D(x=8, y=0), Index2D(x=9, y=0), Index2D(x=10, y=0), Index2D(x=11, y=0), Index2D(x=12, y=0), Index2D(x=13, y=0), Index2D(x=14, y=0), Index2D(x=15, y=0), Index2D(x=16, y=0), Index2D(x=17, y=0), Index2D(x=18, y=0), Index2D(x=19, y=0), Index2D(x=20, y=0), Index2D(x=21, y=0), Index2D(x=0, y=1), Index2D(x=1, y=1), Index2D(x=2, y=1), Index2D(x=3, y=1), Index2D(x=4, y=1), Index2D(x=5, y=1), Index2D(x=6, y=1), Index2D(x=7, y=1), Index2D(x=8, y=1), Index2D(x=9, y=1), Index2D(x=10, y=1), Index2D(x=11, y=1), Index2D(x=12, y=1), Index2D(x=13, y=1), Index2D(x=14, y=1), Index2D(x=15, y=1), Index2D(x=16, y=1), Index2D(x=17, y=1), Index2D(x=18, y=1), Index2D(x=19, y=1), Index2D(x=20, y=1), Index2D(x=21, y=1), Index2D(x=0, y=2), Index2D(x=1, y=2), Index2D(x=2, y=2), Index2D(x=3, y=2), Index2D(x=4, y=2), Index2D(x=5, y=2), Index2D(x=6, y=2), Index2

To pick out a single cell from the coadd, we can use any of the `Index2D` indices to pick one out.

In [11]:
example_cell = coadd.cells[Index2D(x=5,y=4)]

To see the number of input warps that went into a cell:

In [12]:
cell_inputs = example_cell.inputs
print(len(cell_inputs))

3


## Showing cell images

The backend used for displaying image data will be firefly here, though other options are available. 

Some other points:
- Define both `display1` and `display2` in the same notebook cell each time, to keep the displays updated and avoid loading image data from previous notebook cells.
- Firefly will load another tab within the notebook environment. Keep this tab open, since as far as I know the only way to get this tab back is to reload the notebook environment. Redefining the displays will update the current tab instead of creating a new tab.

In [13]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('firefly')

The image data stored in a cell can be called in the following way: 

In [14]:
cell_imMask_out = coadd.cells[Index2D(x=0, y=0)].outer.asMaskedImage()
cell_imMask_in = coadd.cells[Index2D(x=0, y=0)].inner.asMaskedImage()

The inner cell is the boundary of just the single cell, whereas the outer cell contains overlapping image data with neighboring cells. To display the difference between those images, call the below notebook cell:

In [15]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(cell_imMask_out)
display2 = afwDisplay.Display(frame=2)
display2.mtv(cell_imMask_in)

We can also call on the `stitch_coadd` object to display all cells across an entire patch:

In [16]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(stitch_coadd.asExposure())

### Cells and input warps

Another way to visualize the cell information may be to plot both the cell image and a cutout of an input warp that aligns with the cell.

First pick a cell and see what input warps go into it.

In [17]:
cell_warps = list(coadd.cells[cell_list[0]].inputs)
cell_visits = [input.visit for input in cell_warps]
print(cell_visits)

[1242, 1248, 19680, 19684, 19694, 19696, 30490]


We can see that the example warp loaded in earlier is also an input for this cell, so we can use that one.

In [18]:
bbox = coadd.cells[Index2D(x=11,y=11)].outer.bbox
masked_warp = warp[bbox].getMaskedImage()

# display the cell and warp cutout side by side
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.cells[Index2D(x=11,y=11)].outer.image)
display2 = afwDisplay.Display(frame=2)
display2.mtv(masked_warp.image)

### List of cells with a specific visit

Let's find all the cells that use a specific visit. First, query the butler for all possible visits for this patch.

In [19]:
visits = []

for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         physical_filter='HSC-I', 
                                         collections=collection, 
                                         instrument='HSC', 
                                         tract=9813, 
                                         skymap = 'hsc_rings_cells_v1', 
                                         patch=61):
    visits.append(ref.dataId.get('visit'))

visits = np.unique(visits)
print(visits)

[ 1242  1248 19680 19684 19694 19696 30482 30490]


Now we have the list of all possible visits for these cells. Let's just pick the first one.

In [20]:
test_visit = 1242

In [21]:
cell_list_test = [index for index in cell_list if (test_visit in [input.visit for input in list(coadd.cells[index].inputs)])]

In [22]:
print(len(cell_list_test))
print(cell_list_test)

396
[Index2D(x=0, y=0), Index2D(x=1, y=0), Index2D(x=2, y=0), Index2D(x=3, y=0), Index2D(x=4, y=0), Index2D(x=5, y=0), Index2D(x=6, y=0), Index2D(x=7, y=0), Index2D(x=8, y=0), Index2D(x=9, y=0), Index2D(x=10, y=0), Index2D(x=11, y=0), Index2D(x=12, y=0), Index2D(x=13, y=0), Index2D(x=14, y=0), Index2D(x=15, y=0), Index2D(x=16, y=0), Index2D(x=17, y=0), Index2D(x=18, y=0), Index2D(x=19, y=0), Index2D(x=20, y=0), Index2D(x=21, y=0), Index2D(x=0, y=1), Index2D(x=1, y=1), Index2D(x=2, y=1), Index2D(x=3, y=1), Index2D(x=4, y=1), Index2D(x=5, y=1), Index2D(x=6, y=1), Index2D(x=7, y=1), Index2D(x=8, y=1), Index2D(x=9, y=1), Index2D(x=10, y=1), Index2D(x=11, y=1), Index2D(x=12, y=1), Index2D(x=13, y=1), Index2D(x=14, y=1), Index2D(x=15, y=1), Index2D(x=16, y=1), Index2D(x=17, y=1), Index2D(x=18, y=1), Index2D(x=19, y=1), Index2D(x=20, y=1), Index2D(x=21, y=1), Index2D(x=0, y=2), Index2D(x=1, y=2), Index2D(x=2, y=2), Index2D(x=3, y=2), Index2D(x=4, y=2), Index2D(x=5, y=2), Index2D(x=6, y=2), In

## Getting mask information from cells

### For a single cell

To see what masks are being used in a cell, load in the mask plane of an example cell.

In [23]:
cell_mask = coadd.cells[Index2D(x=0, y=0)].inner.mask

In [24]:
print(cell_mask.getMaskPlaneDict())

{'BAD': 0, 'CR': 3, 'CROSSTALK': 9, 'DETECTED': 5, 'DETECTED_NEGATIVE': 6, 'EDGE': 4, 'INTRP': 2, 'NOT_DEBLENDED': 10, 'NO_DATA': 8, 'SAT': 1, 'STREAK': 11, 'SUSPECT': 7, 'UNMASKEDNAN': 12}


If you have the mask plane name, find what value is associated with it

In [25]:
print(cell_mask.getMaskPlane('CR'))

3


Get the number of pixels with a specific mask within a cell

In [26]:
mask_used = 'CR'
print("Mask used: ", mask_used)
mask_bit = cell_mask.getPlaneBitMask(mask_used)
print("Mask bit number: ", mask_bit)
num_bad_pixels = len(np.where(np.bitwise_and(cell_mask.array, mask_bit))[0])
print("Number of masked pixels: ", num_bad_pixels)

Mask used:  CR
Mask bit number:  8
Number of masked pixels:  244


### For a whole patch

In [27]:
mask_used = 'CR'
print("Mask used: ", mask_used)
mask_bit = stitch_coadd.mask.getPlaneBitMask(mask_used)
print("Mask bit number: ", mask_bit)
num_bad_pixels = len(np.where(np.bitwise_and(stitch_coadd.mask.array, mask_bit))[0])
print("Number of masked pixels: ", num_bad_pixels)

Mask used:  CR
Mask bit number:  8
Number of masked pixels:  84878


### Plot the mask plane

See the warp and example cell mask planes.

In [28]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.cells[Index2D(x=11,y=11)].outer.mask)
display2 = afwDisplay.Display(frame=2)
display2.mtv(masked_warp.mask)